In [5]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle


In [7]:
data = pd.read_csv('../data/VesselSimulator/feature1.csv')
data.head()

,Dati,Time,DEPTH,ENGINE_1_FLOWTEMPA,ENGINE_1_FUEL_CONSUMPTION,ENGINE_2_FLOWTEMPA,ENGINE_2_FUEL_CONSUMPTION,HEADING,LATITUDE,LONGITUDE,...,snowfall,weathercode,is_weekday,wind_force,wind_direc,effective_wind_factor,effective_wind,resist_ratio1,resist_ratio2,adversarial
0,190830_022700,147.0,0.0049,25.6000,332.9910,25.6000,334.3725,176.4,49.1936,-123.9554,...,0.0,1,True,20.241001,2.0,0.954240,5.346895,0.002412,0.0,0
1,190830_022800,148.0,0.0037,25.6083,401.2981,25.6000,383.3828,179.7,49.1943,-123.9554,...,0.0,1,True,96.012562,1.0,0.178802,0.641006,0.001969,0.0,0
2,190830_022900,149.0,0.0158,25.7000,423.8546,25.6417,424.2150,186.9,49.1965,-123.9555,...,0.0,1,True,278.439282,0.0,-0.969872,-5.510522,0.001711,0.0,0
3,190830_023000,150.0,0.0206,25.7867,561.3356,25.7100,536.0195,215.1,49.1994,-123.9543,...,0.0,1,True,445.691210,0.0,-0.992757,-7.372911,0.001434,0.0,0
4,190830_023100,151.0,0.0215,25.9300,648.2187,25.8250,615.8354,232.2,49.2024,-123.9504,...,0.0,1,True,1002.317608,0.0,-0.949972,-14.854428,0.001337,0.0,0


In [5]:
data.columns

Index(['Dati', 'Time', 'DEPTH', 'ENGINE_1_FLOWTEMPA',
       'ENGINE_1_FUEL_CONSUMPTION', 'ENGINE_2_FLOWTEMPA',
       'ENGINE_2_FUEL_CONSUMPTION', 'HEADING', 'LATITUDE', 'LONGITUDE',
       'PITCH_1', 'PITCH_2', 'POWER_1', 'POWER_2', 'SOG', 'SOG_SPEEDLOG_LONG',
       'SOG_SPEEDLOG_TRANS', 'SPEED_1', 'SPEED_2', 'STW', 'THRUST_1',
       'THRUST_2', 'TORQUE_1', 'TORQUE_2', 'WIND_ANGLE', 'WIND_SPEED',
       'WIND_ANGLE_TRUE', 'WIND_SPEED_TRUE', 'trip_id', 'MODE', 'datetime',
       'season', 'weekday', 'current', 'direction', 'pressure', 'rain',
       'snowfall', 'weathercode', 'is_weekday', 'wind_force', 'wind_direc',
       'effective_wind_factor', 'effective_wind', 'resist_ratio1',
       'resist_ratio2', 'adversarial'],
      dtype='object')

In [6]:
# remove Time, Depth column
X = data.drop(['Time', 'DEPTH', 'Dati', 'TORQUE_1', 'TORQUE_2', 'THRUST_2', 'THRUST_1', 'SOG_SPEEDLOG_LONG', 'SOG_SPEEDLOG_TRANS'], axis=1)

# create a new column as ENGINE_1_FUEL_CONSUMPTION + ENGINE_2_FUEL_CONSUMPTION
X['SFC'] = X['ENGINE_1_FUEL_CONSUMPTION'] + X['ENGINE_2_FUEL_CONSUMPTION']
X['SPEED'] = X['SPEED_1'] + X['SPEED_2']
X['ENGINE_FLOWTEMPA'] = X['ENGINE_1_FLOWTEMPA'] + X['ENGINE_2_FLOWTEMPA']
X['PITCH'] = X['PITCH_1'] + X['PITCH_2']
X['POWER'] = X['POWER_1'] + X['POWER_2']
X['resist_ratio'] = X['resist_ratio1'] + X['resist_ratio2']


# filter only adverserial is 0
X = X[X['adversarial'] == 0]

X = X.drop(['ENGINE_1_FUEL_CONSUMPTION','ENGINE_2_FUEL_CONSUMPTION', 'adversarial', 'wind_direc', 'SPEED_1', 'SPEED_1',
            'ENGINE_1_FLOWTEMPA', 'ENGINE_2_FLOWTEMPA', 'PITCH_1', 'PITCH_2',
            'POWER_1', 'POWER_2', 'resist_ratio1', 'resist_ratio2'], axis=1)

# convert direction to 1 if N-H other 0
X['direction'] = X['direction'].apply(lambda x: 1 if x == 'N-H' else 0)

# shift the 'SFC', 'SOG', 'LATITUDE', 'LONGITUDE' one time to create previous output 
X['SFC_p'] = X['SFC'].shift(1)
X['SOG_p'] = X['SOG'].shift(1)
X['LATITUDE_p'] = X['LATITUDE'].shift(1)
X['LONGITUDE_p'] = X['LONGITUDE'].shift(1)

# for the first row of each trip_id, fill with the current values. the first two rows in the *_p are the same
# TODO

# bring the SPEED, HEADING, MODE to the front
X = X[['SPEED', 'HEADING', 'MODE', # action space
        'ENGINE_FLOWTEMPA', 'PITCH', 'POWER', 'STW','WIND_ANGLE', 
        'WIND_SPEED', 'WIND_ANGLE_TRUE', 'WIND_SPEED_TRUE',
        'season', 'weekday', 'current', 'direction',
       'pressure', 'rain', 'snowfall', 'weathercode', 'is_weekday',
       'wind_force', 'effective_wind_factor', 'effective_wind', 'resist_ratio',
       'SFC_p', 'SOG_p', 'LATITUDE_p', 'LONGITUDE_p', # previous outputs
        'SFC', 'SOG', 'LATITUDE', 'LONGITUDE', # output (to be removed)
        'trip_id', 'datetime', # to be removed
       ]]



In [7]:
# for each trip_id group, find the first 3 rows and last 3 rows of LATITUDE and LONGITUDE and return the mean for each group
X['LATITUDE_start'] = X.groupby('trip_id')['LATITUDE'].transform(lambda x: x.iloc[:3].mean() )
X['LATITUDE_end'] = X.groupby('trip_id')['LATITUDE'].transform(lambda x: x.iloc[-3:].mean())
X['LONGITUDE_start'] = X.groupby('trip_id')['LONGITUDE'].transform(lambda x: x.iloc[:3].mean() )
X['LONGITUDE_end'] = X.groupby('trip_id')['LONGITUDE'].transform(lambda x: x.iloc[-3:].mean())

# group by direction
LATITUDE_goal_1 = X.groupby('direction')['LATITUDE_start'].mean() 
LATITUDE_goal_2 = X.groupby('direction')['LATITUDE_end'].mean() 

LONGITUDE_goal_1 = X.groupby('direction')['LONGITUDE_start'].mean()
LONGITUDE_goal_2 = X.groupby('direction')['LONGITUDE_end'].mean()

LATITUDE_goal_0 = LATITUDE_goal_1.iloc[0] + LATITUDE_goal_2.iloc[1]
LATITUDE_goal_1 = LATITUDE_goal_1.iloc[1] + LATITUDE_goal_2.iloc[0]
LONGITUDE_goal_0 = LONGITUDE_goal_1.iloc[0] + LONGITUDE_goal_2.iloc[1]
LONGITUDE_goal_1 = LONGITUDE_goal_1.iloc[1] + LONGITUDE_goal_2.iloc[0]

X.drop(['LATITUDE_start', 'LATITUDE_end', 'LONGITUDE_start', 'LONGITUDE_end'], axis=1, inplace=True)

print(LATITUDE_goal_0, LATITUDE_goal_1, LONGITUDE_goal_0, LONGITUDE_goal_1)



98.7576705724596 98.39082973454371 -246.54262885559197 -247.9102732632956


In [8]:
# compute average duration of the trip for each direction
X.groupby(['trip_id', 'direction'])['direction'].count().groupby('direction').max()



direction
0    119
1    143
Name: direction, dtype: int64

In [9]:
# create new column as start_trip and set it to True for the first elemet of trip_id that changes otherwise False
X['start_trip'] = X['trip_id'] != X['trip_id'].shift(1)
# to be removed when training the model

In [10]:
X = X.drop(['trip_id'], axis=1)

In [16]:
# compute the correlation to the output and then rank them from the highest to the lowest
X = X.drop([ 'datetime'], axis=1)
correlation = X.corr()
cor = (correlation['SOG']+ correlation['SFC'] + correlation['LATITUDE'] + correlation['LONGITUDE']).abs()
cor.sort_values(ascending=True)

resist_ratio             0.004694
rain                     0.005810
snowfall                 0.021537
weekday                  0.024909
season                   0.031160
pressure                 0.032282
PITCH                    0.032494
weathercode              0.040977
is_weekday               0.043238
ENGINE_FLOWTEMPA         0.078914
direction                0.125468
WIND_SPEED_TRUE          0.125503
wind_force               0.176978
WIND_ANGLE_TRUE          0.191317
WIND_ANGLE               0.262224
WIND_SPEED               0.333473
effective_wind_factor    0.339515
effective_wind           0.398461
current                  0.502031
HEADING                  0.526809
start_trip               0.685255
SPEED                    0.900819
MODE                     1.211084
SOG_p                    1.575276
SOG                      1.639247
STW                      1.660278
SFC_p                    1.687722
POWER                    1.695855
SFC                      1.702485
LATITUDE_p    

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
correlation = X[['pressure', 'weathercode','is_weekday', 'WIND_SPEED_TRUE','WIND_ANGLE_TRUE', 'WIND_ANGLE','WIND_SPEED','effective_wind_factor' ,'effective_wind', 'wind_force' ]].corr().abs()
# compute the sum of each row 
correlation['sum'] = correlation.sum(axis=1)
print(correlation['sum'].sort_values(ascending=False))
# show with heatmap
# make a big figure
plt.figure(figsize=(10, 10))
sns.heatmap(correlation, annot=True)

NameError: name 'X' is not defined

In [28]:
correlation = X[['WIND_SPEED_TRUE','WIND_ANGLE_TRUE', 'WIND_ANGLE','WIND_SPEED','effective_wind_factor' ,'SOG', 'SFC', 'LATITUDE', 'LONGITUDE' ]].corr()
cor = (correlation['SOG']+ correlation['SFC'] + correlation['LATITUDE'] + correlation['LONGITUDE']).abs()
cor.sort_values(ascending=True)

WIND_SPEED_TRUE          0.125503
WIND_ANGLE_TRUE          0.191317
WIND_ANGLE               0.262224
WIND_SPEED               0.333473
effective_wind_factor    0.339515
SOG                      1.639247
SFC                      1.702485
LATITUDE                 1.834236
LONGITUDE                1.917613
dtype: float64

In [225]:
# convert to two df, one for data before 2021-01-01, one for data after 2021-01-01
X1 = X[X['datetime'] < '2021-01-01']
X2 = X[X['datetime'] >= '2021-01-01']

In [226]:
# create Y as SFC, SOG, LATITUDE, and LONGITUDE
Y1 = X1[['SFC', 'SOG', 'LATITUDE', 'LONGITUDE']]
X1 = X1.drop(['SFC', 'SOG', 'LATITUDE', 'LONGITUDE', 'datetime'], axis=1)
Y2 = X2[['SFC', 'SOG', 'LATITUDE', 'LONGITUDE']]
X2 = X2.drop(['SFC', 'SOG', 'LATITUDE', 'LONGITUDE', 'datetime'], axis=1)

In [227]:
X1.columns

Index(['SPEED', 'HEADING', 'MODE', 'ENGINE_FLOWTEMPA', 'PITCH', 'POWER', 'STW',
       'WIND_ANGLE', 'WIND_SPEED', 'WIND_ANGLE_TRUE', 'WIND_SPEED_TRUE',
       'season', 'weekday', 'current', 'direction', 'pressure', 'rain',
       'snowfall', 'weathercode', 'is_weekday', 'wind_force',
       'effective_wind_factor', 'effective_wind', 'resist_ratio',
       'start_trip'],
      dtype='object')

In [228]:
# save a start_trip dataframe for all data that start_trip is True for X1 and X2 and corresponding Y1 and Y2
X1_start_trip = X1[X1['start_trip'] == True]
X2_start_trip = X2[X2['start_trip'] == True]
Y1_start_trip = Y1[X1['start_trip'] == True]
Y2_start_trip = Y2[X2['start_trip'] == True]
X1_start_trip = X1_start_trip.drop(['start_trip'], axis=1)
X2_start_trip = X2_start_trip.drop(['start_trip'], axis=1)

# remove start_trip column
X1 = X1.drop(['start_trip'], axis=1)
X2 = X2.drop(['start_trip'], axis=1)

In [229]:
# standardize the data with min-max scaler
scaler_X = MinMaxScaler()
# create augmented X1, X2 dataframe
df_aug = pd.concat([X1, X2], ignore_index=True)
scaler_X.fit(df_aug)
X1 = scaler_X.transform(X1)
X2 = scaler_X.transform(X2)

# do the same for Y
scaler_Y = MinMaxScaler()
df_aug = pd.concat([Y1, Y2], ignore_index=True)
scaler_Y.fit(df_aug)
Y1 = scaler_Y.transform(Y1)
Y2 = scaler_Y.transform(Y2)


In [230]:
# transform to numpy array

X1_start_trip = scaler_X.transform(X1_start_trip)
X2_start_trip = scaler_X.transform(X2_start_trip)
Y1_start_trip = scaler_Y.transform(Y1_start_trip)
Y2_start_trip = scaler_Y.transform(Y2_start_trip)

start_trip = {'X1': X1_start_trip, 'X2': X2_start_trip, 'Y1': Y1_start_trip, 'Y2': Y2_start_trip}

# save to pickle start_trip
with open('../data/start_trip.pkl', 'wb') as f:
    pickle.dump(start_trip, f)

scaler = {'scaler_X': scaler_X, 'scaler_Y': scaler_Y}
# save the scaler_X and scaler_Y
with open('../data/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)




In [231]:
# assume the X1 and Y1 are the training data. try a polynomial regression model order 2

# Create a polynomial regression model
degree = 2  # You can change the degree as needed
model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Fit the model
model.fit(X1, Y1)



Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [232]:
# test on X2, Y2
Y2_pred = model.predict(X2)

# calculate the mean squared error
mean_squared_error(Y2, Y2_pred)


0.04782697551744096

In [233]:
# create a new model based on Y2, X2
model2 = make_pipeline(PolynomialFeatures(degree), LinearRegression())
model2.fit(X2, Y2)




Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [234]:
# save model1 and model2 as pickle file

with open('../data/model_2019_2020.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('../data/model_2021.pkl', 'wb') as f:
    pickle.dump(model2, f)